<a href="https://colab.research.google.com/github/chitkalaakella/Coding-with-AI/blob/main/Planetary_Ray_Angle_Calculation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import importlib
import sys

# Check if astropy is installed
if importlib.util.find_spec('astropy') is None:
    print("Error: The Astropy library is required to run this code.")
    print("Please install it using pip:")
    print("  " + sys.executable + " -m pip install astropy")
    sys.exit()

from astropy.coordinates import get_body, EarthLocation, SkyCoord
from astropy.time import Time
import astropy.units as u
from astropy.coordinates import solar_ephemeris, AffineTransform
from scipy.spatial.transform import Rotation as R

def get_planet_earth_heeq(planet_name, time):
    """
    Gets the Heliocentric Earth Equatorial (HEEQ) coordinates of a planet and Earth.

    Args:
        planet_name (str): Name of the planet.
        time (Time): Astropy Time object for the desired time.

    Returns:
        tuple: (planet_heeq, earth_heeq) as astropy CartesianRepresentation objects,
               or (None, None) if a planet name is invalid.
    """
    if planet_name.lower() == 'earth':
        planet_loc = EarthLocation.of_site('greenwich')
        planet_pos = get_body('earth', time, location=planet_loc)
        earth_pos = planet_pos  # Earth's HEEQ is relative to the Sun
    else:
        planet_loc = EarthLocation.of_site('greenwich')
        planet_pos = get_body(planet_name, time, location=planet_loc)
        earth_pos = get_body('earth', time, location=planet_loc)

    # 1. Get heliocentric ecliptic coordinates.
    planet_ecliptic = planet_pos. HeliocentricEcliptic
    earth_ecliptic = earth_pos. HeliocentricEcliptic

    # 2. Define the transformation to HEEQ.  This requires several rotations.
    #    See https://www.mssl.ucl.ac.uk/grid/iau/extra/local_copy/SP_coords/helitran.htm
    #    and https://en.wikipedia.org/wiki/Heliocentric_Earth_equatorial_coordinate_system

    # a. Rotation about Z-axis to align with the Sun's ecliptic longitude.
    sun_lon = solar_ephemeris.get_sun(time). HeliocentricEcliptic.lon
    rot_z_lon = R.from_euler('z', sun_lon.deg, degrees=True)

    # b. Rotation about X-axis by the obliquity of the ecliptic (tilt of Earth's axis).
    obliquity = 23.4397 * u.deg  # Approximate value, can be time-dependent for better accuracy
    rot_x_obliquity = R.from_euler('x', obliquity.deg, degrees=True)

    # c. Rotation about Z-axis by the longitude of the ascending node of the solar equator
    #    on the ecliptic.  This and the solar equator inclination define the orientation
    #    of the Sun's equator.
    solar_equator_asc_node = 75.7 * u.deg  # Approximate value.
    rot_z_node = R.from_euler('z', solar_equator_asc_node.deg, degrees=True)

    # d. Rotation about X-axis by the inclination of the solar equator to the ecliptic.
    solar_equator_inclination = 7.25 * u.deg  # Approximate value
    rot_x_inclination = R.from_euler('x', solar_equator_inclination.deg, degrees=True)
    # Combine the rotations.  The order is important.
    # The order of rotations is: 1. align with sun's longitude, 2. tilt to ecliptic,
    # 3. rotate to solar node, 4. tilt to solar equator.
    heeq_matrix = rot_x_inclination.as_matrix() @ rot_z_node.as_matrix() @ rot_x_obliquity.as_matrix() @ rot_z_lon.as_matrix()
    to_heeq = AffineTransform(matrix=heeq_matrix)

    planet_heeq = planet_ecliptic.transform_to(to_heeq)
    earth_heeq = earth_ecliptic.transform_to(to_heeq)

    return planet_heeq.cartesian.xyz.to(u.AU).value, earth_heeq.cartesian.xyz.to(u.AU).value


def calculate_intersection_angle(planet_pos_heeq, earth_pos_heeq, earth_intersect_point_heeq,
                               planet_face_center, time):
    """
    Calculates the angle between the ray from the center of a planet's face to a
    point on Earth's equator and the Earth's equatorial plane at that point.
    This is done in the HEEQ frame.

    Args:
        planet_pos_heeq (np.ndarray): Planet's HEEQ coordinates (x, y, z) in AU.
        earth_pos_heeq (np.ndarray): Earth's HEEQ coordinates (x, y, z) in AU.
        earth_intersect_point_heeq (np.ndarray): Point on Earth's surface in HEEQ (x,y,z) in AU.
        planet_face_center (np.ndarray):  The center point of the planet's face
                                          relative to the planet's center, in the same
                                          HEEQ coordinates as planet_pos_heeq.
        time (Time): The time at which the calculation is done.

    Returns:
        float: Angle in degrees, or None if vectors are zero.
    """

    # Calculate the ray vector from the center of the planet's face.
    ray_vector = earth_intersect_point_heeq - (planet_pos_heeq + planet_face_center)
    earth_center_heeq = earth_pos_heeq

    # 1.  Find the normal vector to the Earth's surface at the intersection point.
    surface_normal = (earth_intersect_point_heeq - earth_center_heeq)
    if np.linalg.norm(surface_normal) == 0:
        return None
    unit_normal = surface_normal / np.linalg.norm(surface_normal)

    # 2. Define Earth's equatorial plane in HEEQ.
    #    In HEEQ, the Z-axis is aligned with the Sun's rotation axis.
    #    We need to find a vector that is perpendicular to Earth's equatorial plane
    #    *at the location of the intersection point*.
    #    This is NOT necessarily the same as the HEEQ Z-axis.
    #
    #    To do this correctly, we need to use the Earth's rotation at the given time.
    #    We can get this from Astropy.  We'll use the ITRS frame, and then transform
    #    a vector from that frame to HEEQ.

    earth_location = EarthLocation.from_vector(earth_intersect_point_heeq * u.AU)  # Create EarthLocation

    # Get the unit vector representing the Z-axis in the ITRS frame
    itrs_z_axis = np.array([0, 0, 1])

    # Create a SkyCoord object representing the Z-axis in ITRS
    itrs_z_coord = SkyCoord(x=itrs_z_axis[0], y=itrs_z_axis[1], z=itrs_z_axis[2], frame='itrs', obstime=time)
    # Transform this vector to HEEQ
    icrs_z_coord = itrs_z_coord.transform_to('icrs')
    heeq_z_coord = icrs_z_coord.cartesian.xyz.value

    # Normalize the vector.
    unit_equatorial_normal = heeq_z_coord / np.linalg.norm(heeq_z_coord)

    # 3. Calculate the angle.
    dot_product_surface = np.dot(ray_vector, unit_normal)
    angle_with_surface_rad = np.arccos(dot_product_surface / (np.linalg.norm(ray_vector) * np.linalg.norm(unit_normal)))
    angle_with_surface_deg = np.degrees(angle_with_surface_rad)
    angle_from_surface = 90.0 - angle_with_surface_deg

    dot_product_equator = np.dot(ray_vector, unit_equatorial_normal)
    angle_with_equatorial_rad = np.arccos(
        dot_product_equator / (np.linalg.norm(ray_vector) * np.linalg.norm(unit_equatorial_normal)))
    angle_with_equatorial_deg = np.degrees(angle_with_equatorial_rad)

    return angle_with_equatorial_deg  # , angle_from_surface # Return both angles


# --- Main Execution ---
if __name__ == "__main__":
    planet_name = "mars"
    time = Time("2025-04-22T12:00:00", format='isot', scale='utc')

    planet_heeq, earth_heeq = get_planet_earth_heeq(planet_name, time)

    if planet_heeq is not None and earth_heeq is not None:
        print(f"HEEQ position of {planet_name}: {planet_heeq} AU")
        print(f"HEEQ position of Earth: {earth_heeq} AU")

        # Find a point on Earth's equator.  We'll start by finding a vector
        # from the Earth's center to a point on the equator.  We'll use
        # a longitude of 0 for simplicity.
        earth_radius_au = 6371 / 149597870.7  # Earth radius in AU
        # In ITRS, the equator is in the XY plane.  We need to find a point
        # on the equator and then transform that point into HEEQ.
        # Start with a point at longitude 0, latitude 0 in ITRS.
        itrs_equator_point = EarthLocation.from_geodetic(lon=0 * u.deg, lat=0 * u.deg, height=0 * u.m,
                                                        obstime=time)
        equator_point_icrs = SkyCoord(itrs_equator_point.get_vector(time=time), frame='itrs')

        # Transform to HEEQ
        equator_point_heeq_coord = equator_point_icrs.transform_to('icrs')
        equator_point_heeq = get_planet_earth_heeq(planet_name, time)[
                                 1] + equator_point_heeq_coord.cartesian.xyz.to(u.AU).value  # add earth_heeq to get the point relative to the sun.

        # Define the center point of the planet's face.  This is a crucial
        # modification.  For now, let's assume the "face" is a circle
        # pointing directly towards the Sun.  We'll define its center
        # relative to the planet's center.
        planet_radius_au = 3389.5 / 149597870.7 if planet_name.lower() == "mars" else 6051.8/149597870.7 # Radius of Mars in AU
        planet_face_center = np.array([planet_radius_au, 0, 0])  # Example: Assume face is along +X axis


        angle = calculate_intersection_angle(planet_heeq, earth_heeq, equator_point_heeq,
                                            planet_face_center, time)

        if angle is not None:
            print(
                f"Angle of ray from {planet_name} face center to Earth's equator with Earth's equatorial plane: {angle:.2f} degrees")
        else:
            print("Could not calculate intersection angle.")
    else:
        print("Could not retrieve planet positions.")